# Autoencoder und Generative Models  - Exercise

###  Simple Autoencoder

Erstellen Sie wie in der Vorlesung gezeigt einen Autoencoder für das Encoding der MNIST-Daten. 

- Erstellen dazu zuerst folgende Netzwerkarchitektur:

    - Inputlayer der Größe 784
    - Zwischenlayer mit 100 Neuronen (relu activation)
    - Encoding-Layer/Latent-Space mit 2 Neuronen (linear activation)
    - Zwischenlayer mit 100 Neuronen (relu activation)
    - Outputlayer mit der Größe des Inputlayers

- Kompilieren und Trainieren Sie das Modell mit den MNIST-Daten
- Visualisieren Sie den Latent Space
- Vergleichen Sie Input/output für ein beliebiges Bild

In [ ]:
import keras
from keras.models import Sequential # Sequential model API
from keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, Lambda
from keras.models import Model      # Functional model API
from keras import Input
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(60000,28*28 )
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(10000,28*28 )

In [ ]:
################
# Your code here
################

###  Variational Autoencoder

In dieser Aufgabe sollen mithilfe eines Variational Autoencoders Bilder erzeugt werden:

- Lesen Sie dazu das in der Vorlesung gezeigte Modell ein
- Rastern Sie den Latent Space im Bereich $[-4,4]\times [-4,4]$ auf einem $10\times 10$ Raster und erzeugen dann mithilfe des Decoders Bilder
- Vergleichen Sie Input/Output für ein beliebiges Bild aus dem Trainig-Set

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

In [ ]:
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2  

#########
# ENCODER
#########
input_img = Input(shape=img_shape)

x = Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

encoder=Model(input_img,z)

#########
# DECODER
#########
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(np.prod(shape_before_flattening[1:]),activation='relu')(decoder_input)
x = Reshape(shape_before_flattening[1:])(x)
x = Conv2DTranspose(32, 3,padding='same', activation='relu', strides=(2, 2))(x)
x = Conv2D(1, 3,padding='same', activation='sigmoid')(x)

decoder = Model(decoder_input, x)
z_decoded = decoder(z)

class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

y = CustomVariationalLayer()([input_img, z_decoded])

vae = Model(input_img, y)
vae.load_weights('vae_weights.h5')

In [ ]:
################
# Your code here
################